In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# 1. Evironment Setup

In [1]:
!pip install -q transformers datasets rouge-score bert-score


  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.9 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 17.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.3 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 82.1 MB/s eta 0:00:00:00:0100:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependen

In [2]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
import torch


2025-05-12 15:00:38.363263: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747062038.587889      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747062038.658397      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
from transformers import pipeline

# Load a summarization pipeline
summarizer = pipeline("summarization", model="sshleifer/distilbart-cnn-12-6")

# Test the summarizer on dummy text
text = """The Eiffel Tower is a wrought-iron lattice tower on the Champ de Mars in Paris, France. 
It is named after the engineer Gustave Eiffel, whose company designed and built the tower."""
summary = summarizer(text, max_length=45, min_length=10, do_sample=False)

print(summary[0]['summary_text'])


config.json:   0%|          | 0.00/1.80k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

Device set to use cuda:0


 The Eiffel Tower is a wrought-iron lattice tower on the Champ de Mars in Paris, France . It is named after the engineer Gustave Eiffel, whose company designed and built the


# 2. Dataset Preprocessing

In [4]:
from datasets import load_dataset

# Load the CNN/DailyMail dataset
dataset = load_dataset("cnn_dailymail", "3.0.0")

# Show a sample article and summary
print("Sample article:\n", dataset['train'][0]['article'][:500])
print("\nSample summary:\n", dataset['train'][0]['highlights'])


README.md:   0%|          | 0.00/15.6k [00:00<?, ?B/s]

train-00000-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

train-00001-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

train-00002-of-00003.parquet:   0%|          | 0.00/259M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/34.7M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/30.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/287113 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13368 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11490 [00:00<?, ? examples/s]

Sample article:
 LONDON, England (Reuters) -- Harry Potter star Daniel Radcliffe gains access to a reported £20 million ($41.1 million) fortune as he turns 18 on Monday, but he insists the money won't cast a spell on him. Daniel Radcliffe as Harry Potter in "Harry Potter and the Order of the Phoenix" To the disappointment of gossip columnists around the world, the young actor says he has no plans to fritter his cash away on fast cars, drink and celebrity parties. "I don't plan to be one of those people who, as s

Sample summary:
 Harry Potter star Daniel Radcliffe gets £20M fortune as he turns 18 Monday .
Young actor says he has no plans to fritter his cash away .
Radcliffe's earnings from first five Potter films have been held in trust fund .


In [1]:
from transformers import AutoTokenizer
from datasets import load_dataset

# Dataset names and versions to loop over
dataset_versions = {
    "cnn_dailymail": "3.0.0",  # Specify version for CNN/DailyMail
    "xsum": None,  # No version needed for XSum
    "arxiv": None,  # No version needed for Arxiv
    "pubmed": None  # No version needed for PubMed
}

# Model names to loop over
model_names = {
    "t5": "t5-small",
    "bart": "facebook/bart-large-cnn",
    "pegasus": "google/pegasus-xsum"
}

# Define max lengths
max_input_length = 1024
max_target_length = 128

# Store tokenized datasets
tokenized_datasets = {}

# Loop over dataset names
for dataset_key, dataset_version in dataset_versions.items():
    print(f"Loading dataset: {dataset_key} version {dataset_version}")
    
    # Load the dataset with the specified version (if required)
    if dataset_key == "cnn_dailymail":
        dataset = load_dataset(dataset_key, dataset_version)
    else:
        dataset = load_dataset(dataset_key)

    # Tokenize the dataset for each model
    for model_key, model_name in model_names.items():
        tokenizer = AutoTokenizer.from_pretrained(model_name)
        
        def preprocess_function(examples):
            # Select inputs and targets
            if dataset_key == "cnn_dailymail":
                inputs = examples["article"]
                targets = examples["highlights"]
            elif dataset_key == "xsum":
                inputs = examples["document"]
                targets = examples["summary"]
            elif dataset_key in ["arxiv", "pubmed"]:
                inputs = examples["abstract"]
                targets = examples["abstract"]  # Using abstract-based summaries
                
            # Adjust for T5 formatting
            if "t5" in model_name:
                inputs = [f"summarize: {doc}" for doc in inputs]
            
            model_inputs = tokenizer(
                inputs, max_length=max_input_length, truncation=True, padding="max_length"
            )
            
            with tokenizer.as_target_tokenizer():
                labels = tokenizer(
                    targets, max_length=max_target_length, truncation=True, padding="max_length"
                )
            
            model_inputs["labels"] = labels["input_ids"]
            return model_inputs
        
        # Tokenizing and storing the result
        print(f"Tokenizing for {model_name} on {dataset_key}...")
        tokenized = dataset.map(
            preprocess_function,
            batched=True,
            remove_columns=["id"]  # Remove only the 'id' column which exists in cnn_dailymail dataset
        )
        
        # Save the tokenized dataset for each model and dataset combination
        if dataset_key not in tokenized_datasets:
            tokenized_datasets[dataset_key] = {}
        
        tokenized_datasets[dataset_key][model_key] = tokenized


Loading dataset: cnn_dailymail version 3.0.0


README.md:   0%|          | 0.00/15.6k [00:00<?, ?B/s]

train-00000-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

train-00001-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

train-00002-of-00003.parquet:   0%|          | 0.00/259M [00:00<?, ?B/s]

...

Map:   0%|          | 0/204045 [00:00<?, ? examples/s]

Map:   0%|          | 0/11332 [00:00<?, ? examples/s]

Map:   0%|          | 0/11334 [00:00<?, ? examples/s]

Loading dataset: arxiv version None


DatasetNotFoundError: Dataset 'arxiv' doesn't exist on the Hub or cannot be accessed.

In [2]:
from datasets import load_dataset, load_from_disk, DatasetDict
from transformers import AutoTokenizer
import os

# Models and tokenizers
model_names = {
    "t5": "t5-small",
    "bart": "facebook/bart-base",
    "pegasus": "google/pegasus-xsum"
}
tokenizers = {name: AutoTokenizer.from_pretrained(path) for name, path in model_names.items()}

# Preprocessing function
def preprocess_function(example, tokenizer):
    inputs = example["article"]
    targets = example["highlights"]
    model_inputs = tokenizer(inputs, max_length=512, truncation=True)
    labels = tokenizer(targets, max_length=128, truncation=True)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs


config.json:   0%|          | 0.00/1.72k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [12]:
def preprocess_function(example, tokenizer):
    # Dynamically detect source/target keys
    if "article" in example and "highlights" in example:
        inputs = example["article"]
        targets = example["highlights"]
    elif "document" in example and "summary" in example:
        inputs = example["document"]
        targets = example["summary"]
    elif "text" in example and "summary" in example:
        inputs = example["text"]
        targets = example["summary"]
    elif "article" in example and "abstract" in example:
        inputs = example["article"]
        targets = example["abstract"]
    else:
        raise ValueError("Unknown dataset schema. Expected article/highlights, document/summary, or text/summary.")

    model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding="max_length")
    labels = tokenizer(targets, max_length=128, truncation=True, padding="max_length")
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs


In [3]:
from datasets import load_dataset, load_from_disk
import os

base_path = "/kaggle/working/tokenized"
dataset_name = "cnn_dailymail"
config = "3.0.0"
dataset = load_dataset(dataset_name, config)

for model_key, tokenizer in tokenizers.items():
    save_path = f"{base_path}/{dataset_name}_{config}/{model_key}"
    if os.path.exists(save_path):
        print(f"[{model_key.upper()}] Tokenized CNN/DailyMail exists at {save_path}, loading...")
        tokenized = load_from_disk(save_path)
    else:
        print(f"[{model_key.upper()}] Tokenizing CNN/DailyMail...")
        tokenized = dataset.map(lambda x: preprocess_function(x, tokenizer), batched=True, remove_columns=dataset["train"].column_names)
        tokenized.save_to_disk(save_path)
        print(f"[{model_key.upper()}] Saved to {save_path}")


[T5] Tokenizing CNN/DailyMail...


Map:   0%|          | 0/287113 [00:00<?, ? examples/s]

Map:   0%|          | 0/13368 [00:00<?, ? examples/s]

Map:   0%|          | 0/11490 [00:00<?, ? examples/s]

Saving the dataset (0/2 shards):   0%|          | 0/287113 [00:00<?, ? examples/s]

...

Map:   0%|          | 0/11490 [00:00<?, ? examples/s]

Saving the dataset (0/2 shards):   0%|          | 0/287113 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/13368 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/11490 [00:00<?, ? examples/s]

[PEGASUS] Saved to /kaggle/working/tokenized/cnn_dailymail_3.0.0/pegasus


In [6]:
dataset_name = "xsum"
dataset = load_dataset(dataset_name, trust_remote_code=True)

for model_key, tokenizer in tokenizers.items():
    save_path = f"{base_path}/{dataset_name}/{model_key}"
    if os.path.exists(save_path):
        print(f"[{model_key.upper()}] Tokenized XSum exists at {save_path}, loading...")
        tokenized = load_from_disk(save_path)
    else:
        print(f"[{model_key.upper()}] Tokenizing XSum...")
        tokenized = dataset.map(lambda x: preprocess_function(x, tokenizer), batched=True, remove_columns=dataset["train"].column_names)
        tokenized.save_to_disk(save_path)
        print(f"[{model_key.upper()}] Saved to {save_path}")


[T5] Tokenizing XSum...


Map:   0%|          | 0/204045 [00:00<?, ? examples/s]

Map:   0%|          | 0/11332 [00:00<?, ? examples/s]

Map:   0%|          | 0/11334 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/204045 [00:00<?, ? examples/s]

...

Map:   0%|          | 0/11334 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/204045 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/11332 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/11334 [00:00<?, ? examples/s]

[PEGASUS] Saved to /kaggle/working/tokenized/xsum/pegasus


In [7]:
dataset_name = "scientific_papers"
config = "pubmed"
dataset = load_dataset(dataset_name, config)

for model_key, tokenizer in tokenizers.items():
    save_path = f"{base_path}/{dataset_name}_{config}/{model_key}"
    if os.path.exists(save_path):
        print(f"[{model_key.upper()}] Tokenized PubMed exists at {save_path}, loading...")
        tokenized = load_from_disk(save_path)
    else:
        print(f"[{model_key.upper()}] Tokenizing PubMed...")
        tokenized = dataset.map(lambda x: preprocess_function(x, tokenizer), batched=True, remove_columns=dataset["train"].column_names)
        tokenized.save_to_disk(save_path)
        print(f"[{model_key.upper()}] Saved to {save_path}")


README.md:   0%|          | 0.00/8.27k [00:00<?, ?B/s]

scientific_papers.py:   0%|          | 0.00/5.35k [00:00<?, ?B/s]

The repository for scientific_papers contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/scientific_papers.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N]  y


Generating train split:   0%|          | 0/119924 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/6633 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/6658 [00:00<?, ? examples/s]

[T5] Tokenizing PubMed...


Map:   0%|          | 0/119924 [00:00<?, ? examples/s]

ValueError: Unknown dataset schema. Expected article/highlights or document/summary.

In [13]:
for model_key, tokenizer in tokenizers.items():
    save_path = f"{base_path}/{dataset_name}_{config}/{model_key}"
    if os.path.exists(save_path):
        print(f"[{model_key.upper()}] Tokenized PubMed exists at {save_path}, loading...")
        tokenized = load_from_disk(save_path)
    else:
        print(f"[{model_key.upper()}] Tokenizing PubMed...")
        tokenized = dataset.map(lambda x: preprocess_function(x, tokenizer), batched=True, remove_columns=dataset["train"].column_names)
        tokenized.save_to_disk(save_path)
        print(f"[{model_key.upper()}] Saved to {save_path}")

[T5] Tokenizing PubMed...


Map:   0%|          | 0/119924 [00:00<?, ? examples/s]

Map:   0%|          | 0/6633 [00:00<?, ? examples/s]

Map:   0%|          | 0/6658 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/119924 [00:00<?, ? examples/s]

...

Map:   0%|          | 0/6658 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/119924 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/6633 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/6658 [00:00<?, ? examples/s]

[PEGASUS] Saved to /kaggle/working/tokenized/scientific_papers_pubmed/pegasus


In [1]:
import os
from datasets import load_from_disk
from random import randint

# Map datasets and their paths
datasets_paths = {
    "cnn_dailymail": "/kaggle/working/tokenized/cnn_dailymail_3.0.0",
    "pubmed": "/kaggle/working/tokenized/scientific_papers_pubmed",
    "xsum": "/kaggle/working/tokenized/xsum"
}

# List of models
models = ["bart", "pegasus", "t5"]

# Function to display sample
def show_sample(tokenized, tokenizer, dataset_name, model_name):
    sample = tokenized["train"][randint(0, len(tokenized["train"]) - 1)]
    input_text = tokenizer.decode(sample["input_ids"], skip_special_tokens=True)
    target_text = tokenizer.decode(sample["labels"], skip_special_tokens=True)

    print(f"\n=== {dataset_name.upper()} | {model_name.upper()} ===")
    print("Input:\n", input_text)
    print("\nTarget:\n", target_text)
    print("=" * 50)

# Loop over datasets and models
for dataset_name, dataset_path in datasets_paths.items():
    for model_name in models:
        model_path = os.path.join(dataset_path, model_name)
        if not os.path.exists(model_path):
            print(f"Missing: {model_path}")
            continue

        # Load tokenizer (adjust per model)
        if model_name == "bart":
            from transformers import BartTokenizer
            tokenizer = BartTokenizer.from_pretrained("facebook/bart-large")
        elif model_name == "pegasus":
            from transformers import PegasusTokenizer
            tokenizer = PegasusTokenizer.from_pretrained("google/pegasus-xsum")
        elif model_name == "t5":
            from transformers import T5Tokenizer
            tokenizer = T5Tokenizer.from_pretrained("t5-base")

        # Load tokenized dataset and show sample
        tokenized = load_from_disk(model_path)
        show_sample(tokenized, tokenizer, dataset_name, model_name)


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.63k [00:00<?, ?B/s]

...

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565



=== CNN_DAILYMAIL | T5 ===
Input:
 (CNN) -- You'll find his name and initials everywhere you go at the French Open -- the famous green and brick red logo stamped on everything from parasols and caps to towels and tennis balls. But ask most people who Roland Garros was and they'll tend to double fault. "For a lot of people in France and abroad, Garros is a tennis man, or a sometime president of the French Tennis Federation (FFT)," says Michael Guittard, head of collections and cultural mediation at the FFT. "Everybody knows the name of Roland Garros, but nobody knows who he was. That's why we have an exhibition to explain that he is not a tennis champion, but a hero of World War One." Garros' elevated status wasn't earned down in the trenches, but up in the skies above the Western Front, as the exhibition at the FFT museum entitled "Moi... Roland Garros" explains. Born in 1888 on the French island of Runion in the Indian Ocean, Garros had won fame before the war, notably becoming the f

# 3. Model training

In [48]:
import torch

# Example after training epoch or at specific points
torch.cuda.empty_cache()


In [1]:
from transformers import (
    T5ForConditionalGeneration,
    T5TokenizerFast,
    Trainer,
    TrainingArguments,
    DataCollatorForSeq2Seq,
    logging
)
from datasets import load_from_disk
import torch
from tqdm.auto import tqdm

# Set logging to show HuggingFace progress
logging.set_verbosity_info()

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

# Base paths to tokenized datasets
base_paths = {
    "cnn_dailymail": "/kaggle/working/tokenized/cnn_dailymail_3.0.0/t5",
    "xsum": "/kaggle/working/tokenized/xsum/t5",
    "pubmed": "/kaggle/working/tokenized/scientific_papers_pubmed/t5"
}

# Toggle this to True for fast debugging
DEBUG_MODE = False  # Set to True to limit dataset for fast testing

# Percentage of the dataset to use (for faster training)
PERCENTAGE = 0.1  

# Load tokenizer
tokenizer = T5TokenizerFast.from_pretrained("t5-base")

for dataset_name, path in base_paths.items():
    print(f"\nFine-tuning T5 on {dataset_name}...")

    print(" Loading dataset...")
    dataset = load_from_disk(path)

    # Limit the dataset size using a percentage
    num_samples = int(len(dataset["train"]) * PERCENTAGE)
    dataset["train"] = dataset["train"].select(range(num_samples))

    # Optionally, limit the validation set as well (to the same percentage)
    if "validation" in dataset:
        dataset["validation"] = dataset["validation"].select(range(int(len(dataset["validation"]) * PERCENTAGE)))

    print(f"Dataset loaded: {len(dataset['train'])} training samples")

    # Load model
    print("Loading T5 model...")
    model = T5ForConditionalGeneration.from_pretrained("t5-base").to(device)

    # Use a data collator that dynamically pads inputs
    data_collator = DataCollatorForSeq2Seq(
        tokenizer=tokenizer,
        model=model,
        padding=True  # This replaces padding_strategy
    )

    # Define training arguments
    training_args = TrainingArguments(
        output_dir=f"./results_{dataset_name}",
        num_train_epochs=1,
        per_device_train_batch_size=4,
        per_device_eval_batch_size=4,
        save_steps=500,
        save_total_limit=1,
        logging_steps=100,
        eval_steps=500,
        logging_dir=f"./logs_{dataset_name}",
        report_to="none"
    )

    # Set eval dataset fallback
    eval_ds = dataset["validation"] if "validation" in dataset else dataset["train"].select(range(100))

    # Manual early stopping parameters
    patience = 2  # Number of evaluations without improvement before stopping
    best_loss = float("inf")  # Initialize with a very large number
    no_improvement_count = 0  # Counter for consecutive evaluations without improvement

    # Set up trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=dataset["train"],
        eval_dataset=eval_ds,
        tokenizer=tokenizer,
        data_collator=data_collator
    )

    print("Starting training...")
    for epoch in range(training_args.num_train_epochs):
        trainer.train()

        # Evaluation after each epoch
        eval_results = trainer.evaluate(eval_dataset=eval_ds)

        # Extract the validation loss from the evaluation results
        eval_loss = eval_results["eval_loss"]

        print(f"Epoch {epoch + 1} | Validation Loss: {eval_loss}")

        # Early stopping logic
        if eval_loss < best_loss:
            best_loss = eval_loss
            no_improvement_count = 0
            print(f"Validation loss improved to {eval_loss}. Saving model.")
            model.save_pretrained(f"./t5_{dataset_name}_finetuned")
        else:
            no_improvement_count += 1
            print(f"No improvement in validation loss for {no_improvement_count} evaluations.")

        if no_improvement_count >= patience:
            print("Early stopping triggered!")
            break

    print(f"Training finished for {dataset_name}.")


2025-05-13 10:13:53.200699: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747131233.446719      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747131233.518914      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Using device: cuda


spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

loading file spiece.model from cache at /root/.cache/huggingface/hub/models--t5-base/snapshots/a9723ea7f1b39c1eae772870f3b547bf6ef7e6c1/spiece.model
loading file tokenizer.json from cache at /root/.cache/huggingface/hub/models--t5-base/snapshots/a9723ea7f1b39c1eae772870f3b547bf6ef7e6c1/tokenizer.json
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at None
loading file tokenizer_config.json from cache at None
loading file chat_template.jinja from cache at None


...

Saving model checkpoint to ./results_pubmed/checkpoint-500
Configuration saved in ./results_pubmed/checkpoint-500/config.json
Configuration saved in ./results_pubmed/checkpoint-500/generation_config.json
Model weights saved in ./results_pubmed/checkpoint-500/model.safetensors
tokenizer config file saved in ./results_pubmed/checkpoint-500/tokenizer_config.json
Special tokens file saved in ./results_pubmed/checkpoint-500/special_tokens_map.json
Copy vocab file to ./results_pubmed/checkpoint-500/spiece.model
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
Saving model checkpoint to ./results_pubmed/checkpoint-1000
Configuration saved in ./results_pubmed/checkpoint-1000/config.json
Configuration saved in ./results_pubmed/checkpoint-1000/generation_config.json
Model weights saved in ./results_pubmed/checkpoint-1000/m

Configuration saved in ./t5_pubmed_finetuned/config.json
Configuration saved in ./t5_pubmed_finetuned/generation_config.json


Epoch 1 | Validation Loss: 1.9766994714736938
Validation loss improved to 1.9766994714736938. Saving model.


Model weights saved in ./t5_pubmed_finetuned/model.safetensors


Training finished for pubmed.


In [2]:
import shutil

# Directory to zip
dir_to_zip1 = '/kaggle/working/t5_cnn_dailymail_finetuned'
dir_to_zip2 = '/kaggle/working/t5_pubmed_finetuned'
dir_to_zip3 = '/kaggle/working/t5_xsum_finetuned'

# Output zip file name (without .zip extension)
output_filename1 = 't5_cnn_dailymail_finetuned'
output_filename2 = 't5_pubmed_finetuned'
output_filename3 = 't5_xsum_finetuned'


# Create the zip file
shutil.make_archive(output_filename1, 'zip', dir_to_zip1)
shutil.make_archive(output_filename2, 'zip', dir_to_zip2)
shutil.make_archive(output_filename3, 'zip', dir_to_zip3)

print("Zipping complete.")


Zipping complete.


In [10]:
from datasets import load_from_disk, concatenate_datasets
from transformers import BartForConditionalGeneration, BartTokenizer, Trainer, TrainingArguments, DataCollatorForSeq2Seq

# ========== CONFIG ==========
dataset_paths = [
    '/kaggle/working/tokenized/cnn_dailymail_3.0.0/bart',
    '/kaggle/working/tokenized/scientific_papers_pubmed/bart',
    '/kaggle/working/tokenized/xsum/bart'
]

train_percent = 10  # Train on 10% of each dataset
model_checkpoint = 'facebook/bart-base'
output_dir = './bart_finetuned_all'
num_train_epochs = 1
batch_size = 4

# ========== LOAD & SAMPLE DATASETS ==========
sampled_datasets = []
for path in dataset_paths:
    dataset = load_from_disk(path)['train']
    
    # Sample the desired percentage
    if train_percent < 100:
        sample_size = int(len(dataset) * (train_percent / 100))
        dataset = dataset.shuffle(seed=42).select(range(sample_size))
    
    sampled_datasets.append(dataset)

# ========== CONCATENATE ALL DATASETS ==========
full_train_dataset = concatenate_datasets(sampled_datasets)

# ========== LOAD MODEL & TOKENIZER ==========
tokenizer = BartTokenizer.from_pretrained(model_checkpoint)
model = BartForConditionalGeneration.from_pretrained(model_checkpoint)

# ========== TRAINING ARGUMENTS ==========
training_args = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=batch_size,
    num_train_epochs=num_train_epochs,
    logging_dir='./logs',
    logging_steps=100,
    save_steps=500,
    save_total_limit=2,
    #evaluation_strategy="no",
    report_to="none"
)

# ========== DATA COLLATOR ==========
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

# ========== TRAINER ==========
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=full_train_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator
)

# ========== TRAIN ==========
trainer.train()

# ========== SAVE MODEL ==========
model.save_pretrained('./bart_finetuned_model')
tokenizer.save_pretrained('./bart_finetuned_model')


loading file vocab.json from cache at /root/.cache/huggingface/hub/models--facebook--bart-base/snapshots/aadd2ab0ae0c8268c7c9693540e9904811f36177/vocab.json
loading file merges.txt from cache at /root/.cache/huggingface/hub/models--facebook--bart-base/snapshots/aadd2ab0ae0c8268c7c9693540e9904811f36177/merges.txt
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at None
loading file tokenizer_config.json from cache at None
loading file tokenizer.json from cache at /root/.cache/huggingface/hub/models--facebook--bart-base/snapshots/aadd2ab0ae0c8268c7c9693540e9904811f36177/tokenizer.json
loading file chat_template.jinja from cache at None
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--facebook--bart-base/snapshots/aadd2ab0ae0c8268c7c9693540e9904811f36177/config.json
Model config BartConfig {
  "activation_dropout": 0.1,
  "activation_function": "gelu",
  "add_bias_logits": false,
  "add_final_layer_n

Step,Training Loss
100,3.253600
200,2.790500
300,2.729100
400,2.742600
500,2.660200
600,2.612200
700,2.598400
800,2.617400
900,2.545900
1000,2.572300


Saving model checkpoint to ./bart_finetuned_all/checkpoint-500
/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:3339: UserWarning: Moving the following attributes in the config to the generation config: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(
Configuration saved in ./bart_finetuned_all/checkpoint-500/config.json
Configuration saved in ./bart_finetuned_all/checkpoint-500/generation_config.json
Model weights saved in ./bart_finetuned_all/checkpoint-500/model.safetensors
tokenizer config file saved in ./bart_finetuned_all/checkpoint-500/tokenizer_config.json
Special tokens file saved in ./bart_finetuned_all/checkpoint-500/special_tokens_map.json
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 

('./bart_finetuned_model/tokenizer_config.json',
 './bart_finetuned_model/special_tokens_map.json',
 './bart_finetuned_model/vocab.json',
 './bart_finetuned_model/merges.txt',
 './bart_finetuned_model/added_tokens.json')

In [12]:
import shutil

# Directory to zip
dir_to_zip1 = '/kaggle/working/bart_finetuned_model'
#dir_to_zip2 = '/kaggle/working/t5_pubmed_finetuned'
#dir_to_zip3 = '/kaggle/working/t5_xsum_finetuned'

# Output zip file name (without .zip extension)
output_filename1 = 'bart_finetuned_model'
#output_filename2 = 't5_pubmed_finetuned'
#output_filename3 = 't5_xsum_finetuned'


# Create the zip file
shutil.make_archive(output_filename1, 'zip', dir_to_zip1)
#shutil.make_archive(output_filename2, 'zip', dir_to_zip2)
#shutil.make_archive(output_filename3, 'zip', dir_to_zip3)

print("Zipping complete.")

Zipping complete.


In [12]:
from datasets import load_from_disk, concatenate_datasets
from transformers import PegasusForConditionalGeneration, PegasusTokenizer, Trainer, TrainingArguments, DataCollatorForSeq2Seq

# ========== CONFIG ==========
dataset_paths = [
    '/kaggle/working/tokenized/cnn_dailymail_3.0.0/pegasus',
    '/kaggle/working/tokenized/scientific_papers_pubmed/pegasus',
    '/kaggle/working/tokenized/xsum/pegasus'
]

train_percent = 10  # Train on 10% of each dataset
model_checkpoint = 'google/pegasus-xsum'  # You can also try 'google/pegasus-cnn_dailymail'
output_dir = './pegasus_finetuned_all'
num_train_epochs = 1
batch_size = 2  # Reduce if OOM

# ========== LOAD & SAMPLE DATASETS ==========
sampled_datasets = []
for path in dataset_paths:
    dataset = load_from_disk(path)['train']
    
    # Sample the desired percentage
    if train_percent < 100:
        sample_size = int(len(dataset) * (train_percent / 100))
        dataset = dataset.shuffle(seed=42).select(range(sample_size))
    
    sampled_datasets.append(dataset)

# ========== CONCATENATE ALL DATASETS ==========
full_train_dataset = concatenate_datasets(sampled_datasets)

# ========== LOAD MODEL & TOKENIZER ==========
tokenizer = PegasusTokenizer.from_pretrained(model_checkpoint)
model = PegasusForConditionalGeneration.from_pretrained(model_checkpoint)

# ========== TRAINING ARGUMENTS ==========
training_args = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=batch_size,
    num_train_epochs=num_train_epochs,
    logging_dir='./logs',
    logging_steps=100,
    save_steps=500,
    save_total_limit=2,
    report_to="none"
)

# ========== DATA COLLATOR ==========
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

# ========== TRAINER ==========
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=full_train_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator
)

# ========== TRAIN ==========
trainer.train()

# ========== SAVE MODEL ==========
model.save_pretrained('./pegasus_finetuned_model')
tokenizer.save_pretrained('./pegasus_finetuned_model')


Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-xsum and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipykernel_31/58672280.py:52: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Step,Training Loss
100,2.664000
200,2.335500
300,2.375300
400,2.078100


/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:3339: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 64, 'num_beams': 8, 'length_penalty': 0.6}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


RuntimeError: [enforce fail at inline_container.cc:603] . unexpected pos 3064387840 vs 3064387732

In [1]:
import shutil

# Zip the directory (will create models_backup.zip)
shutil.make_archive('pegasus_finetuned_all', 'zip', '/kaggle/working/pegasus_finetuned_all')


'/kaggle/working/pegasus_finetuned_all.zip'

In [8]:
import os

file_path = "/kaggle/working/t5_xsum_finetuned.zip"
if os.path.exists(file_path):
    os.remove(file_path)
    print("File deleted.")
else:
    print("File not found.")


File deleted.


In [11]:
import os

def get_folder_size(folder):
    total_size = 0
    for dirpath, dirnames, filenames in os.walk(folder):
        for f in filenames:
            fp = os.path.join(dirpath, f)
            if os.path.isfile(fp):
                total_size += os.path.getsize(fp)
    return total_size

folder = "/kaggle/working/t5_pubmed_finetuned"
size_bytes = get_folder_size(folder)
size_mb = size_bytes / (1024 * 1024)
print(f"Folder size: {size_mb:.2f} MB")


Folder size: 850.34 MB


In [6]:
import shutil
import os

# Replace with the path to the folder you want to delete
folder_path = "/kaggle/working/t5_cnn_dailymail_finetuned"

# Check if the folder exists
if os.path.exists(folder_path):
    shutil.rmtree(folder_path)
    print(f"Deleted folder: {folder_path}")
else:
    print(f"Folder not found: {folder_path}")


NotADirectoryError: [Errno 20] Not a directory: '/kaggle/working/t5_pubmed_finetuned.zip'

In [2]:
import torch

torch.cuda.empty_cache()


# 4. Evaluation

# BART

In [11]:
from datasets import load_from_disk
import evaluate
import torch
from tqdm import tqdm
import numpy as np

# Load evaluation metrics
rouge = evaluate.load("rouge")
bertscore = evaluate.load("bertscore")

def evaluate_tokenized_dataset(model, tokenizer, dataset, sample_percent=1):
    """
    Evaluate a summarization model on a tokenized HuggingFace dataset using ROUGE and BERTScore.
    """
    # Move model to appropriate device
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    model.eval()

    # Sample the dataset if sample_percent < 100
    if sample_percent < 100:
        sample_size = int(len(dataset) * (sample_percent / 100))
        dataset = dataset.select(range(sample_size))

    predictions = []
    references = []

    for example in tqdm(dataset, desc="Evaluating"):
        input_ids = torch.tensor(example["input_ids"]).unsqueeze(0).to(device)
        attention_mask = torch.tensor(example["attention_mask"]).unsqueeze(0).to(device)

        with torch.no_grad():
            output_ids = model.generate(
                input_ids=input_ids,
                attention_mask=attention_mask,
                max_length=142,
                num_beams=4,
                early_stopping=True
            )

        # Decode generated and reference summaries
        pred = tokenizer.decode(output_ids[0], skip_special_tokens=True)
        ref = tokenizer.decode(example["labels"], skip_special_tokens=True)

        predictions.append(pred)
        references.append(ref)

    # Compute ROUGE scores
    rouge_result = rouge.compute(predictions=predictions, references=references)
    print("🔸 ROUGE Scores:")
    for key, value in rouge_result.items():
        print(f"{key}: {value:.4f}")

    # Compute BERTScore (F1)
    bert_result = bertscore.compute(predictions=predictions, references=references, lang="en")
    avg_bertscore = np.mean(bert_result["f1"])
    print(f"\n🔹 BERTScore (F1): {avg_bertscore:.4f}")


In [12]:
from transformers import BartTokenizer, BartForConditionalGeneration

# Load model
model = BartForConditionalGeneration.from_pretrained('/kaggle/input/bart-finetuned-model')
tokenizer = BartTokenizer.from_pretrained('/kaggle/input/bart-finetuned-model')

# Load tokenized test dataset
tokenized_dataset = load_from_disk("/kaggle/working/tokenized/cnn_dailymail_3.0.0/bart")["test"]

# Run evaluation
evaluate_tokenized_dataset(model, tokenizer, tokenized_dataset, sample_percent=1)


Evaluating: 100%|██████████| 114/114 [01:11<00:00,  1.59it/s]


🔸 ROUGE Scores:
rouge1: 0.3172
rouge2: 0.1302
rougeL: 0.2365
rougeLsum: 0.2919


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



🔹 BERTScore (F1): 0.8762


In [13]:
from transformers import BartTokenizer, BartForConditionalGeneration

# Load model
model = BartForConditionalGeneration.from_pretrained('/kaggle/input/bart-finetuned-model')
tokenizer = BartTokenizer.from_pretrained('/kaggle/input/bart-finetuned-model')

# Load tokenized test dataset
tokenized_dataset = load_from_disk("/kaggle/working/tokenized/scientific_papers_pubmed/bart")["test"]

# Run evaluation
evaluate_tokenized_dataset(model, tokenizer, tokenized_dataset, sample_percent=1)


Evaluating: 100%|██████████| 66/66 [01:08<00:00,  1.04s/it]


🔸 ROUGE Scores:
rouge1: 0.3525
rouge2: 0.1390
rougeL: 0.2449
rougeLsum: 0.3073

🔹 BERTScore (F1): 0.8540


In [15]:
from transformers import BartTokenizer, BartForConditionalGeneration

# Load model
model = BartForConditionalGeneration.from_pretrained('/kaggle/input/bart-finetuned-model')
tokenizer = BartTokenizer.from_pretrained('/kaggle/input/bart-finetuned-model')

# Load tokenized test dataset
tokenized_dataset = load_from_disk("/kaggle/working/tokenized/xsum/bart")["test"]

# Run evaluation
evaluate_tokenized_dataset(model, tokenizer, tokenized_dataset, sample_percent=1)


Evaluating: 100%|██████████| 113/113 [00:26<00:00,  4.20it/s]


🔸 ROUGE Scores:
rouge1: 0.3439
rouge2: 0.1239
rougeL: 0.2767
rougeLsum: 0.2772

🔹 BERTScore (F1): 0.8991


# T5

In [18]:
from datasets import load_from_disk
import torch
from tqdm import tqdm
import evaluate
import numpy as np

# ========== LOAD METRICS ==========
rouge = evaluate.load("rouge")
bertscore = evaluate.load("bertscore")

# ========== EVALUATION FUNCTION ==========
def evaluate_tokenized_dataset(model, tokenizer, dataset, sample_percent=1):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    model.eval()

    if sample_percent < 100:
        sample_size = int(len(dataset) * (sample_percent / 100))
        dataset = dataset.select(range(sample_size))

    predictions = []
    references = []

    for example in tqdm(dataset, desc="Evaluating"):
        input_ids = torch.tensor(example["input_ids"]).unsqueeze(0).to(device)
        attention_mask = torch.tensor(example["attention_mask"]).unsqueeze(0).to(device)

        with torch.no_grad():
            output_ids = model.generate(
                input_ids=input_ids,
                attention_mask=attention_mask,
                max_length=142,
                num_beams=4,
                early_stopping=True
            )

        pred = tokenizer.decode(output_ids[0], skip_special_tokens=True)
        ref = tokenizer.decode(example["labels"], skip_special_tokens=True)

        predictions.append(pred)
        references.append(ref)

    # ROUGE
    rouge_result = rouge.compute(predictions=predictions, references=references)
    print("🔸 ROUGE Scores:")
    for key, value in rouge_result.items():
        print(f"{key}: {value:.4f}")

    # BERTScore
    bert_result = bertscore.compute(predictions=predictions, references=references, lang="en")
    avg_bertscore = np.mean(bert_result["f1"])
    print(f"\n🔹 BERTScore (F1): {avg_bertscore:.4f}")


In [23]:
from transformers import T5Tokenizer, T5ForConditionalGeneration
from datasets import load_from_disk

# Paths
model_path = "/kaggle/input/t5-cnn-dailymail-finetuned"
tokenized_dataset_path = "/kaggle/working/tokenized/cnn_dailymail_3.0.0/t5"
sample_percent = 5  # Evaluate on 5% of test set

# Load tokenized dataset
dataset = load_from_disk(tokenized_dataset_path)['test']

# Load tokenizer from base model (use "t5-base" if that was your backbone)
tokenizer = T5Tokenizer.from_pretrained("t5-base")

# Load model from fine-tuned checkpoint
model = T5ForConditionalGeneration.from_pretrained(
    model_path, 
    trust_remote_code=True,  # optional, helps when model is saved using safetensors
    #from_safetensors=True    # required since you're using model.safetensors
)

# Run your evaluation function
evaluate_tokenized_dataset(model, tokenizer, dataset, sample_percent=sample_percent)


spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

Evaluating: 100%|██████████| 574/574 [10:43<00:00,  1.12s/it]


🔸 ROUGE Scores:
rouge1: 0.3172
rouge2: 0.1310
rougeL: 0.2359
rougeLsum: 0.2360


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



🔹 BERTScore (F1): 0.8759


In [24]:
from transformers import T5Tokenizer, T5ForConditionalGeneration
from datasets import load_from_disk

# Paths
model_path = "/kaggle/working/t5_pubmed_finetuned"
tokenized_dataset_path = "/kaggle/working/tokenized/scientific_papers_pubmed/t5"
sample_percent = 5  # Evaluate on 5% of test set

# Load tokenized dataset
dataset = load_from_disk(tokenized_dataset_path)['test']

# Load tokenizer from base model (use "t5-base" if that was your backbone)
tokenizer = T5Tokenizer.from_pretrained("t5-base")

# Load model from fine-tuned checkpoint
model = T5ForConditionalGeneration.from_pretrained(
    model_path, 
    trust_remote_code=True,  # optional, helps when model is saved using safetensors
    #from_safetensors=True    # required since you're using model.safetensors
)

# Run your evaluation function
evaluate_tokenized_dataset(model, tokenizer, dataset, sample_percent=sample_percent)


Evaluating: 100%|██████████| 332/332 [14:09<00:00,  2.56s/it]


🔸 ROUGE Scores:
rouge1: 0.3456
rouge2: 0.1352
rougeL: 0.2348
rougeLsum: 0.2348

🔹 BERTScore (F1): 0.8489


In [25]:
from transformers import T5Tokenizer, T5ForConditionalGeneration
from datasets import load_from_disk

# Paths
model_path = "/kaggle/working/t5_xsum_finetuned"
tokenized_dataset_path = "/kaggle/working/tokenized/xsum/t5"
sample_percent = 5  # Evaluate on 5% of test set

# Load tokenized dataset
dataset = load_from_disk(tokenized_dataset_path)['test']

# Load tokenizer from base model (use "t5-base" if that was your backbone)
tokenizer = T5Tokenizer.from_pretrained("t5-base")

# Load model from fine-tuned checkpoint
model = T5ForConditionalGeneration.from_pretrained(
    model_path, 
    trust_remote_code=True,  # optional, helps when model is saved using safetensors
    #from_safetensors=True    # required since you're using model.safetensors
)

# Run your evaluation function
evaluate_tokenized_dataset(model, tokenizer, dataset, sample_percent=sample_percent)


Evaluating: 100%|██████████| 566/566 [05:28<00:00,  1.72it/s]


🔸 ROUGE Scores:
rouge1: 0.3297
rouge2: 0.1135
rougeL: 0.2624
rougeLsum: 0.2626

🔹 BERTScore (F1): 0.8942


# Pegasus

In [31]:
from transformers import PegasusTokenizer, PegasusForConditionalGeneration
from datasets import load_from_disk

# ========== CONFIG ==========
model_path = "/kaggle/working/pegasus_finetuned_all/checkpoint-500"
tokenized_dataset_path = "/kaggle/working/tokenized/cnn_dailymail_3.0.0/pegasus"
sample_percent = 5  # % of dataset to evaluate on

# ========== LOAD DATASET ==========
dataset = load_from_disk(tokenized_dataset_path)['test']

# ========== LOAD TOKENIZER & MODEL ==========
tokenizer = PegasusTokenizer.from_pretrained(model_path, local_files_only=True)
model = PegasusForConditionalGeneration.from_pretrained(
    model_path,
    local_files_only=True,
    #from_safetensors=True  # optional but preferred if using safetensors
)

# ========== RUN EVALUATION ==========
evaluate_tokenized_dataset(model, tokenizer, dataset, sample_percent=sample_percent)


Evaluating: 100%|██████████| 574/574 [10:38<00:00,  1.11s/it]


🔸 ROUGE Scores:
rouge1: 0.2999
rouge2: 0.1185
rougeL: 0.2254
rougeLsum: 0.2256

🔹 BERTScore (F1): 0.8707


In [32]:
from transformers import PegasusTokenizer, PegasusForConditionalGeneration
from datasets import load_from_disk

# ========== CONFIG ==========
model_path = "/kaggle/working/pegasus_finetuned_all/checkpoint-500"
tokenized_dataset_path = "/kaggle/working/tokenized/scientific_papers_pubmed/pegasus"
sample_percent = 5  # % of dataset to evaluate on

# ========== LOAD DATASET ==========
dataset = load_from_disk(tokenized_dataset_path)['test']

# ========== LOAD TOKENIZER & MODEL ==========
tokenizer = PegasusTokenizer.from_pretrained(model_path, local_files_only=True)
model = PegasusForConditionalGeneration.from_pretrained(
    model_path,
    local_files_only=True,
    #from_safetensors=True  # optional but preferred if using safetensors
)

# ========== RUN EVALUATION ==========
evaluate_tokenized_dataset(model, tokenizer, dataset, sample_percent=sample_percent)


Evaluating: 100%|██████████| 332/332 [12:51<00:00,  2.32s/it]


🔸 ROUGE Scores:
rouge1: 0.3305
rouge2: 0.1243
rougeL: 0.2253
rougeLsum: 0.2255

🔹 BERTScore (F1): 0.8440


In [33]:
from transformers import PegasusTokenizer, PegasusForConditionalGeneration
from datasets import load_from_disk

# ========== CONFIG ==========
model_path = "/kaggle/working/pegasus_finetuned_all/checkpoint-500"
tokenized_dataset_path = "/kaggle/working/tokenized/xsum/pegasus"
sample_percent = 5  # % of dataset to evaluate on

# ========== LOAD DATASET ==========
dataset = load_from_disk(tokenized_dataset_path)['test']

# ========== LOAD TOKENIZER & MODEL ==========
tokenizer = PegasusTokenizer.from_pretrained(model_path, local_files_only=True)
model = PegasusForConditionalGeneration.from_pretrained(
    model_path,
    local_files_only=True,
    #from_safetensors=True  # optional but preferred if using safetensors
)

# ========== RUN EVALUATION ==========
evaluate_tokenized_dataset(model, tokenizer, dataset, sample_percent=sample_percent)


Evaluating: 100%|██████████| 566/566 [05:16<00:00,  1.79it/s]


🔸 ROUGE Scores:
rouge1: 0.4427
rouge2: 0.2288
rougeL: 0.3738
rougeLsum: 0.3740

🔹 BERTScore (F1): 0.9172


# Sample examples

In [6]:
from transformers import (
    PegasusTokenizer, PegasusForConditionalGeneration,
    T5Tokenizer, T5ForConditionalGeneration,
    BartTokenizer, BartForConditionalGeneration
)
from datasets import load_dataset

# Define model paths (update if needed)
model_paths = {
    "BART": "/kaggle/input/bart-cnn-finetuned",
    "T5_CNN": "/kaggle/input/t5-cnn-dailymail-finetuned",
    "T5_PubMed": "/kaggle/working/t5_pubmed_finetuned",
    "T5_XSum": "/kaggle/working/t5_xsum_finetuned",
    "Pegasus_CNN": "/kaggle/working/pegasus_finetuned_all/checkpoint-500"
}

# Load raw test samples
cnn = load_dataset("cnn_dailymail", "3.0.0", split="test[:1]")
pubmed = load_dataset("scientific_papers", "pubmed", split="test[:1]")
xsum = load_dataset("xsum", split="test[:1]")

# Extract the first article from each dataset
articles = {
    "CNN": cnn[0]["article"],
    "PubMed": pubmed[0]["article"],
    "XSum": xsum[0]["document"]
}



In [9]:
# Define model paths (update if needed)
model_paths = {
    "BART": "/kaggle/input/bart-finetuned-model",
    "T5_CNN": "/kaggle/input/t5-cnn-dailymail-finetuned",
    "T5_PubMed": "/kaggle/working/t5_pubmed_finetuned",
    "T5_XSum": "/kaggle/working/t5_xsum_finetuned",
    "Pegasus_CNN": "/kaggle/working/pegasus_finetuned_all/checkpoint-500"
}

def summarize_article(model, tokenizer, article, max_input_length, max_output_length=200):
    inputs = tokenizer(
        article,
        return_tensors="pt",
        max_length=max_input_length,
        truncation=True  # Important to avoid overlength issues
    )
    summary_ids = model.generate(
        inputs["input_ids"],
        max_length=max_output_length,
        num_beams=4,
        early_stopping=True
    )
    return tokenizer.decode(summary_ids[0], skip_special_tokens=True)


# Load models/tokenizers
models = {
    "BART_CNN": (BartTokenizer.from_pretrained(model_paths["BART"]), BartForConditionalGeneration.from_pretrained(model_paths["BART"])),
    "T5_CNN": (T5Tokenizer.from_pretrained("t5-base"), T5ForConditionalGeneration.from_pretrained(model_paths["T5_CNN"])),
    "T5_PubMed": (T5Tokenizer.from_pretrained("t5-base"), T5ForConditionalGeneration.from_pretrained(model_paths["T5_PubMed"])),
    "T5_XSum": (T5Tokenizer.from_pretrained("t5-base"), T5ForConditionalGeneration.from_pretrained(model_paths["T5_XSum"])),
    "Pegasus_CNN": (PegasusTokenizer.from_pretrained(model_paths["Pegasus_CNN"]), PegasusForConditionalGeneration.from_pretrained(model_paths["Pegasus_CNN"]))
}

for source_name, article in articles.items():
    print(f"\n\n{'='*30} {source_name} Article {'='*30}\n")
    print(article)
    print(f"\n{'-'*30} Summaries {'-'*30}")
    for model_name, (tokenizer, model) in models.items():
        try:
            if "Pegasus" in model_name:
                summary = summarize_article(model, tokenizer, article, max_input_length=512)
            else:
                summary = summarize_article(model, tokenizer, article, max_input_length=1024)
                
            print(f"\n[Summary by {model_name}]:\n{summary}\n")
        except Exception as e:
            print(f"\n[Summary by {model_name}]: Error - {e}\n")



============================== CNN Article ==============================

(CNN)The Palestinian Authority officially became the 123rd member of the International Criminal Court on Wednesday, a step that gives the court jurisdiction over alleged crimes in Palestinian territories. The formal accession was marked with a ceremony at The Hague, in the Netherlands, where the court is based. The Palestinians signed the ICC's founding Rome Statute in January, when they also accepted its jurisdiction over alleged crimes committed "in the occupied Palestinian territory, including East Jerusalem, since June 13, 2014." Later that month, the ICC opened a preliminary examination into the situation in Palestinian territories, paving the way for possible war crimes investigations against Israelis. As members of the court, Palestinians may be subject to counter-charges as well. Israel and the United States, neither of which is an ICC member, opposed the Palestinians' efforts to join the body. But Pal